In [43]:
def vector_sum(u, v):
    return dict((x, u.get(x, 0) + v.get(x, 0)) for x in set(u)|set(v))

def scalar_times_vector(m, u):
    return {x: m*y for x, y in u.items()}

def add_constant_to_entry(a, i, k):
    return tuple(a[j] + k*kronecker_delta(i, j) for j in range(len(a)))

def element_times_vector(m, u):
    return {(m,) + x: y for x, y in u.items()}

def drop_first_entry(u):
    return tuple(u[i] for i in range(1, len(u)))

def Partitions1(n):
    if n < 0:
        return [[]]
    a=list(Partitions(n))
    a.reverse() #al parecer, esto mejora performance en un 12% masomenos, talvez otros ordenes son mejores aun
    return a

def stbasis1(n):
    basis = []
    for i in range(1,n,2):
        basis = basis + [Partitions(n-i)]
    
def vector_simplify(u):
    v = {}
    for a in set(u):
        v = vector_sum(v, {tuple(sorted(a, reverse = True)): sgn(a)*(-1)^((len(a)*(len(a) - 1))/2)*u[a]})
    return {x: y for x, y in v.items() if y!=0}

def translation(u):
    v = {}
    for a in set (u):
        w = {}
        for i in range (len(a)):
            w = vector_sum(w, {add_constant_to_entry(a, i, 1): a[i] + 1})
        w = scalar_times_vector(u[a], w)
        v = vector_sum(v, w)
    return vector_simplify(v)

def sgn (a):
    n = 0
    for i in range(len(a)):
        for j in range(i + 1, len(a)):
            if a[i] == a[j]:
                return 0
            else:
                if a[i] > a[j]:
                    n += 1
    if n % 2 == 0:
        return 1
    else:
        return -1

def virasoro(m, u):
    if m == 1:
        return translation(u)
    else:
        v = {}
        for a in set(u):
            if a == ():
                if m == 2:
                    v = vector_sum(v, {(1, 0): 1/2*u[()]})
                else:
                    v = vector_sum(v, scalar_times_vector(1/(m - 2), translation(virasoro(m - 1, {(): 1}))))                 
            else:
                b = vector_sum({add_constant_to_entry(a, 0, m): (m + 1)/2 + a[0]}, element_times_vector(a[0], virasoro(m, {drop_first_entry(a): 1})))
                v = vector_sum(v, scalar_times_vector(u[a], b))
        return vector_simplify(v)
    
def several_virasoro(a, u):
    v = u
    for i in range(len(a)):
        v = virasoro(a[len(a) - i - 1], v)
    return v

def is_zero_list(a):
    for i in range(len(a)):
        if a[i] != 0:
            return False
    return True

def matrix_has_solution(A):
    A = A.echelon_form()
    for i in range(A.nrows()):
        if A[A.nrows() - 1 - i, A.ncols() - 1] != 0:
            if is_zero_list(A.rows()[A.nrows() - 1 - i][0: A.ncols() - 1]):
                return False
            else:
                return True
    return True

def insert_row(A, k, a):
    return matrix(A.rows()[:k] + [a] + A.rows()[k:])

def insert_column(A, a):
    return (insert_row(A.transpose(), A.ncols(), a)).transpose()

def vector_in_span2(u, S):
    index = tuple(set(u))
    A = matrix({(i, j): S[j][index[i]] for j in range(len(S)) for i in range(len(index))}, sparse = False)
    A = insert_column(A, [u[index[i]] for i in range(len(index))])
    return matrix_has_solution(A)

def vector_in_span(u, S2):
    S = copy(S2)
    index = tuple(set(u))
    for i in range(len(S)):
        index = tuple(set(u).union(set(S[i]))) #AQUI RADICABA EL PROBLEMA, ESTO NO ES TODA LA BASE
    u = dict((x, u.get(x, 0)) for x in set(index))
    for i in range(len(S)):
        S[i] = dict((x, S[i].get(x, 0)) for x in set(index))
    return vector_in_span2(u, S)

def basis0(n):
    for i in range(1, n):
        S = []
        P = []
        for p in Partitions1(i):
            v = several_virasoro(p, {(): 1})
            if not vector_in_span(v, S):
                S = S + [v]
                P = P + [p]
        print([i, P, len(P)])
        
def basis1(n):
    for i in range(1, n):
        S = []
        P = []
        for p in Partitions1(i):
            v = several_virasoro(p, {(0,): 1})
            if not vector_in_span(v, S):
                S = S + [v]
                P = P + [p]
        print([i, P, len(P)])

def multiplicity(a, p):
    n = 0
    for i in range(len(p)):
        if p[i] == a:
            n += 1
    return n

def contains(p, q):
    for a in set(q):
        if multiplicity(a, q) > multiplicity(a, p):
            return False
    return True

def simplify_partitions(P):
    Q = []
    for i in range(len(P)):
        if all([not contains(P[i], Q[j]) for j in range(len(Q))]):
            Q = Q + [P[i]]
    return Q

def missing_partitions0(n):
    P = []
    Q = []
    for i in range(1, n):
        S = []
        for p in list(Partitions(i)):
            v = several_virasoro(p, {(): 1})
            if not vector_in_span(v, S):
                S = S + [v]
                P = P + [list(p)]
    for i in range(1, n):
        for p in list(Partitions(i)):
            Q = Q + [list(p)]
    R = [p for p in Q if p not in P]
    print(simplify_partitions(R))
    
def missing_partitions1(n):
    P = []
    Q = []
    for i in range(1, n):
        S = []
        for p in list(Partitions(i)):
            v = several_virasoro(p, {(0,): 1})
            if not vector_in_span(v, S):
                S = S + [v]
                P = P + [list(p)]
    for i in range(1, n):
        for p in list(Partitions(i)):
            Q = Q + [list(p)]
    R = [p for p in Q if p not in P]
    print(simplify_partitions(R))
    
def verify_basis1_up_to(w, n):
    for i in range(2,n+1):
        S = []
        a = st_basisF1(i)
        A = matrix(0,len(a))
        for j in range(len(w[i])):
            v = several_virasoro(w[i][j], {(0,):1})
            vrow = [v.get(a[k], 0) for k in range(len(a))]
            A = insert_row(A, A.nrows(), vrow)
        print(i)
        print(A)
        print(A.det())
        [print("------")]
        
def verify_basis1_up_tov2(w, n):
    for i in range(n+1):
        S = []
        for j in range(len(w[i])):
            v = several_virasoro(w[i][j], {(0,):1})
            if not vector_in_span(v, S):
                S = S + [v]
        P = []
        #print(S)
        for p in Partitions1(i):
            v = several_virasoro(p, {(0,): 1})
            #print(v)
            #print(S)
            if not vector_in_span(v, S):
                S = S + [v]
                P = P + [list(p)]
        print([i, P, len(P)])

In [8]:
for p in list(Partitions(8, length = 1)):
    print (p)

[8]


In [45]:
verify_basis1_up_to(w, 14)

2
[2]
2
------
3
[2]
2
------
4
[  3  -1]
[5/2 1/2]
4
------
5
[ 7/2 -3/2]
[   3    1]
8
------
6
[  9   0  -3]
[  4  -2   0]
[7/2 3/2 1/2]
-48
------
7
[  11    0    0    1]
[  10    0   -4   -2]
[ 9/2 -5/2    0  1/2]
[   4    2    1    0]
112
------
8
[65/4  9/4 -7/4 15/4  5/2]
[  12    0    0   -4    0]
[  11    0   -5    0   -3]
[   5   -3    0    0    1]
[ 9/2  5/2  3/2  1/2    0]
-3584
------
9
[  21   -5    4    0   -3    3]
[35/2  5/2   -2    0  3/2  5/2]
[  13    0    0   -5    0   -3]
[  12    0   -6    0   -4    0]
[11/2 -7/2    0    0  3/2  1/2]
[   5    3    2    1    0    0]
57344
------
10
[   28    -9     0     6     0     7    -1     0]
[   24     6     0    -4     0     6     6     0]
[ 45/2 -11/2   9/2     0  -7/2 -15/2     0   3/2]
[ 75/4  11/4  -9/4     0   7/4 -25/4     0   5/4]
[   14     0     0    -6     0     0    -4    -2]
[   13     0    -7     0    -5     0     0     1]
[    6    -4     0     0     2     0     1     0]
[ 11/2   7/2   5/2   3/2     0   1/2  

In [37]:
A = matrix([[1,2,3],[4,5,6],[7,8,9]])
A.det()

0

In [10]:
list(Partitions(8, length = 2))

[[7, 1], [6, 2], [5, 3], [4, 4]]

In [23]:
def st_basisF1(n):
    basis = []
    for i in range(1, n, 2):
        for p in list(Partitions(Integer(n+1/2+i-i^2/2), length = i)):
            p = tuple(p[j]+(i-2-j) for j in range(i))
            basis = basis + [p]
    return basis
st_basisF1(10)

[(10,),
 (8, 1, 0),
 (7, 2, 0),
 (6, 3, 0),
 (6, 2, 1),
 (5, 4, 0),
 (5, 3, 1),
 (4, 3, 2)]

In [12]:
for j in range(1,n,2):
    print(j)

TypeError: 'function' object cannot be interpreted as an integer

In [15]:
for i in range(1,7,2):
    print(i)

1
3
5


In [19]:
1/2+5^2/2

13

In [112]:
SS = []
for j in range(len(w[6])):
    vv = several_virasoro(w[6][j], {(0,):1})
    SS = SS + [vv]
print(SS)
uu = several_virasoro((2,1,1,1,1),{(0,):1})
print(uu)
vector_in_span(uu, SS)

[{(6,): 9, (3, 2, 0): -3}, {(6,): 4, (4, 1, 0): -2}, {(6,): 7/2, (4, 1, 0): 3/2, (3, 2, 0): 1/2}]
{(6,): 132, (4, 1, 0): 12}


True

In [121]:
stbasis1(6)

1
3
5


In [132]:
verify_basis1_up_to(w, 20)

[0, [], 0]
[1, [], 0]
[2, [], 0]
[3, [], 0]
[4, [], 0]
[5, [], 0]
[6, [], 0]
[7, [], 0]
[8, [], 0]
[9, [], 0]
[10, [], 0]
[11, [], 0]
[12, [], 0]
[13, [], 0]
[14, [], 0]
[15, [], 0]
[16, [], 0]
[17, [], 0]
[18, [], 0]
[19, [], 0]
[20, [], 0]


In [133]:
verify_basis1_up_to(w, 20)

[0, [], 0]
[1, [], 0]
[2, [], 0]
[3, [], 0]
[4, [], 0]
[5, [], 0]
[6, [], 0]
[7, [], 0]
[8, [], 0]
[9, [], 0]
[10, [], 0]
[11, [], 0]
[12, [], 0]
[13, [], 0]
[14, [], 0]
[15, [], 0]
[16, [], 0]
[17, [], 0]
[18, [], 0]
[19, [], 0]
[20, [], 0]


In [93]:
several_virasoro((1,1),{(2,):1})

{(4,): 12}

In [95]:
several_virasoro((2,1,1,1,1),{(0,):1})

{(6,): 132, (4, 1, 0): 12}

In [89]:
several_virasoro((4,1,1),{(0,):1})

{(6,): 9, (3, 2, 0): -3}

In [90]:
several_virasoro((5,1),{(0,):1})

{(6,): 4, (4, 1, 0): -2}

In [91]:
several_virasoro((6,),{(0,):1})

{(6,): 7/2, (4, 1, 0): 3/2, (3, 2, 0): 1/2}

In [72]:
verify_basis1_up_to(w, 10)

[0, [], 0]
[1, [], 0]
[2, [], 0]
[3, [], 0]
[4, [], 0]
[5, [], 0]
[6, [[2, 1, 1, 1, 1]], 1]
[7, [[2, 1, 1, 1, 1, 1], [3, 1, 1, 1, 1]], 2]
[8, [], 0]
[9, [[2, 2, 1, 1, 1, 1, 1]], 1]
[10, [[2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1]], 3]


In [75]:
several_virasoro((1,1), {(0,):1})

{(2,): 2}

In [77]:
several_virasoro((1,1), {(1,0):1/2})

{(2, 1): 1, (3, 0): 3}

In [80]:
several_virasoro((1,1,1),{(1,0):1/2})

{(3, 1): 6, (4, 0): 12}

In [84]:
several_virasoro((1,1,1,1),{(1,0):1/2})

{(3, 2): 12, (4, 1): 36, (5, 0): 60}

In [87]:
several_virasoro((6,),{():1})

{(3, 2): 1/2, (4, 1): 3/2, (5, 0): 5/2}

In [88]:
several_virasoro((6,),{(0,):1})

{(6,): 7/2, (4, 1, 0): 3/2, (3, 2, 0): 1/2}

In [82]:
several_virasoro((5,),{():1})

{(3, 1): 1, (4, 0): 2}

In [83]:
several_virasoro((5,1),{(0,):1})

{(6,): 4, (4, 1, 0): -2}

In [78]:
several_virasoro((4,1,1),{(0,):1})

{(6,): 9, (3, 2, 0): -3}

In [69]:
several_virasoro((1,), {(1,0):1})

{(2, 0): 2}

In [59]:
w[6]

[[4, 1, 1], [5, 1], [6]]

In [57]:
several_virasoro(w[6][2], {(0,):1})

{(6,): 7/2, (4, 1, 0): 3/2, (3, 2, 0): 1/2}

In [58]:
several_virasoro([2, 1, 1, 1, 1], {(0,):1})

{(6,): 132, (4, 1, 0): 12}

In [38]:
verify_basis1_up_to(w, 15)

[0, [], 0]
[1, [], 0]
[2, [], 0]
[3, [], 0]
[4, [], 0]
[5, [], 0]
[6, [[2, 1, 1, 1, 1]], 1]
[7, [[2, 1, 1, 1, 1, 1], [3, 1, 1, 1, 1]], 2]
[8, [], 0]
[9, [[2, 2, 1, 1, 1, 1, 1]], 1]
[10, [[2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1]], 3]
[11, [[2, 2, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1]], 4]
[12, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1]], 4]
[13, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1, 1, 1, 1, 1, 1]], 7]
[14, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1,

In [36]:
verify_basis1_up_to(w, 20)

[0, [], 0]
[1, [], 0]
[2, [], 0]
[3, [], 0]
[4, [], 0]
[5, [], 0]
[6, [[2, 1, 1, 1, 1]], 1]
[7, [[2, 1, 1, 1, 1, 1], [3, 1, 1, 1, 1]], 2]
[8, [], 0]
[9, [[2, 2, 1, 1, 1, 1, 1]], 1]
[10, [[2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1]], 3]
[11, [[2, 2, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1]], 4]
[12, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1]], 4]
[13, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1, 1, 1, 1, 1, 1]], 7]
[14, [[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1,

In [14]:
verify_basis1_up_to(w, 20)

[{(0,): 1}]
[{(1,): 1}]
[{(2,): 2}]
[{(3,): 2}]
[{(2, 1, 0): -1, (4,): 3}, {(2, 1, 0): 1/2, (4,): 5/2}]
[{(3, 1, 0): -3/2, (5,): 7/2}, {(3, 1, 0): 1, (5,): 3}]
[{(6,): 9, (3, 2, 0): -3}, {(6,): 4, (4, 1, 0): -2}, {(6,): 7/2, (4, 1, 0): 3/2, (3, 2, 0): 1/2}]
[{(3, 2, 1): 1, (7,): 11}, {(3, 2, 1): -2, (7,): 10, (4, 2, 0): -4}, {(5, 1, 0): -5/2, (3, 2, 1): 1/2, (7,): 9/2}, {(5, 1, 0): 2, (7,): 4, (4, 2, 0): 1}]
[{(4, 3, 0): 15/4, (6, 1, 0): 9/4, (8,): 65/4, (5, 2, 0): -7/4, (4, 2, 1): 5/2}, {(8,): 12, (4, 3, 0): -4}, {(8,): 11, (5, 2, 0): -5, (4, 2, 1): -3}, {(6, 1, 0): -3, (8,): 5, (4, 2, 1): 1}, {(4, 3, 0): 1/2, (6, 1, 0): 5/2, (5, 2, 0): 3/2, (8,): 9/2}]
[{(7, 1, 0): -5, (6, 2, 0): 4, (5, 2, 1): -3, (4, 3, 1): 3, (9,): 21}, {(7, 1, 0): 5/2, (6, 2, 0): -2, (5, 2, 1): 3/2, (4, 3, 1): 5/2, (9,): 35/2}, {(4, 3, 1): -3, (9,): 13, (5, 3, 0): -5}, {(6, 2, 0): -6, (5, 2, 1): -4, (9,): 12}, {(7, 1, 0): -7/2, (5, 2, 1): 3/2, (4, 3, 1): 1/2, (9,): 11/2}, {(7, 1, 0): 3, (6, 2, 0): 2, (5, 3, 0): 1,

In [6]:
several_virasoro((5,2),{(0,):1})

{(3, 2, 1): -3/2, (7,): 15/2, (4, 2, 0): -3}

In [7]:
set(several_virasoro((5,2),{(0,):1}))

{(3, 2, 1), (4, 2, 0), (7,)}

In [41]:
w = [[[]], [[1]], [[1, 1]], [[3]], [[3, 1], [4]], [[4, 1], [5]], [[4, 1, 1], [5, 1], [6]], [[4, 3], [5, 1, 1], [6, 1], [7]], [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]], [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]], [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]], [[5, 5, 1], [6, 4, 1], [6, 5], [7, 3, 1], [7, 4], [8, 3], [9, 1, 1], [10, 1], [11]], [[5, 5, 1, 1], [6, 4, 1, 1], [6, 5, 1], [6, 6], [7, 4, 1], [7, 5], [8, 3, 1], [8, 4], [9, 3], [10, 1, 1], [11, 1], [12]], [[6, 4, 3], [6, 5, 1, 1], [6, 6, 1], [7, 4, 1, 1], [7, 5, 1], [7, 6], [8, 4, 1], [8, 5], [9, 3, 1], [9, 4], [10, 3], [11, 1, 1], [12, 1], [13]], [[6, 5, 3], [6, 6, 1, 1], [7, 4, 3], [7, 5, 1, 1], [7, 6, 1], [7, 7], [8, 4, 1, 1], [8, 5, 1], [8, 6], [9, 4, 1], [9, 5], [10, 3, 1], [10, 4], [11, 3], [12, 1, 1], [13, 1], [14]], [[6, 6, 3], [7, 4, 4], [7, 5, 3], [7, 6, 1, 1], [7, 7, 1], [8, 4, 3], [8, 5, 1, 1], [8, 6, 1], [8, 7], [9, 4, 1, 1], [9, 5, 1], [9, 6], [10, 4, 1], [10, 5], [11, 3, 1], [11, 4], [12, 3], [13, 1, 1], [14, 1], [15]], [[6, 6, 3, 1], [7, 5, 3, 1], [7, 5, 4], [7, 6, 3], [7, 7, 1, 1], [8, 4, 4], [8, 5, 3], [8, 6, 1, 1], [8, 7, 1], [8, 8], [9, 4, 3], [9, 5, 1, 1], [9, 6, 1], [9, 7], [10, 4, 1, 1], [10, 5, 1], [10, 6], [11, 4, 1], [11, 5], [12, 3, 1], [12, 4], [13, 3], [14, 1, 1], [15, 1], [16]], [[7, 5, 4, 1], [7, 6, 3, 1], [7, 6, 4], [7, 7, 3], [8, 5, 3, 1], [8, 5, 4], [8, 6, 3], [8, 7, 1, 1], [8, 8, 1], [9, 4, 4], [9, 5, 3], [9, 6, 1, 1], [9, 7, 1], [9, 8], [10, 4, 3], [10, 5, 1, 1], [10, 6, 1], [10, 7], [11, 4, 1, 1], [11, 5, 1], [11, 6], [12, 4, 1], [12, 5], [13, 3, 1], [13, 4], [14, 3], [15, 1, 1], [16, 1], [17]], [[7, 6, 4, 1], [7, 7, 3, 1], [7, 7, 4], [8, 5, 4, 1], [8, 5, 5], [8, 6, 3, 1], [8, 6, 4], [8, 7, 3], [8, 8, 1, 1], [9, 5, 3, 1], [9, 5, 4], [9, 6, 3], [9, 7, 1, 1], [9, 8, 1], [9, 9], [10, 4, 4], [10, 5, 3], [10, 6, 1, 1], [10, 7, 1], [10, 8], [11, 4, 3], [11, 5, 1, 1], [11, 6, 1], [11, 7], [12, 4, 1, 1], [12, 5, 1], [12, 6], [13, 4, 1], [13, 5], [14, 3, 1], [14, 4], [15, 3], [16, 1, 1], [17, 1], [18]], [[7, 6, 4, 1, 1], [7, 7, 4, 1], [8, 5, 5, 1], [8, 6, 4, 1], [8, 6, 5], [8, 7, 3, 1], [8, 7, 4], [8, 8, 3], [9, 5, 4, 1], [9, 5, 5], [9, 6, 3, 1], [9, 6, 4], [9, 7, 3], [9, 8, 1, 1], [9, 9, 1], [10, 5, 3, 1], [10, 5, 4], [10, 6, 3], [10, 7, 1, 1], [10, 8, 1], [10, 9], [11, 4, 4], [11, 5, 3], [11, 6, 1, 1], [11, 7, 1], [11, 8], [12, 4, 3], [12, 5, 1, 1], [12, 6, 1], [12, 7], [13, 4, 1, 1], [13, 5, 1], [13, 6], [14, 4, 1], [14, 5], [15, 3, 1], [15, 4], [16, 3], [17, 1, 1], [18, 1], [19]], [[7, 7, 4, 1, 1], [8, 5, 5, 1, 1], [8, 6, 4, 1, 1], [8, 6, 5, 1], [8, 7, 4, 1], [8, 7, 5], [8, 8, 3, 1], [8, 8, 4], [9, 5, 5, 1], [9, 6, 4, 1], [9, 6, 5], [9, 7, 3, 1], [9, 7, 4], [9, 8, 3], [9, 9, 1, 1], [10, 5, 4, 1], [10, 5, 5], [10, 6, 3, 1], [10, 6, 4], [10, 7, 3], [10, 8, 1, 1], [10, 9, 1], [10, 10], [11, 5, 3, 1], [11, 5, 4], [11, 6, 3], [11, 7, 1, 1], [11, 8, 1], [11, 9], [12, 4, 4], [12, 5, 3], [12, 6, 1, 1], [12, 7, 1], [12, 8], [13, 4, 3], [13, 5, 1, 1], [13, 6, 1], [13, 7], [14, 4, 1, 1], [14, 5, 1], [14, 6], [15, 4, 1], [15, 5], [16, 3, 1], [16, 4], [17, 3], [18, 1, 1], [19, 1], [20]], [[8, 6, 4, 3], [8, 6, 5, 1, 1], [8, 7, 4, 1, 1], [8, 7, 5, 1], [8, 8, 4, 1], [8, 8, 5], [9, 5, 5, 1, 1], [9, 6, 4, 1, 1], [9, 6, 5, 1], [9, 6, 6], [9, 7, 4, 1], [9, 7, 5], [9, 8, 3, 1], [9, 8, 4], [9, 9, 3], [10, 5, 5, 1], [10, 6, 4, 1], [10, 6, 5], [10, 7, 3, 1], [10, 7, 4], [10, 8, 3], [10, 9, 1, 1], [10, 10, 1], [11, 5, 4, 1], [11, 5, 5], [11, 6, 3, 1], [11, 6, 4], [11, 7, 3], [11, 8, 1, 1], [11, 9, 1], [11, 10], [12, 5, 3, 1], [12, 5, 4], [12, 6, 3], [12, 7, 1, 1], [12, 8, 1], [12, 9], [13, 4, 4], [13, 5, 3], [13, 6, 1, 1], [13, 7, 1], [13, 8], [14, 4, 3], [14, 5, 1, 1], [14, 6, 1], [14, 7], [15, 4, 1, 1], [15, 5, 1], [15, 6], [16, 4, 1], [16, 5], [17, 3, 1], [17, 4], [18, 3], [19, 1, 1], [20, 1], [21]], [[8, 6, 5, 3], [8, 7, 4, 3], [8, 7, 5, 1, 1], [8, 8, 4, 1, 1], [8, 8, 5, 1], [9, 6, 4, 3], [9, 6, 5, 1, 1], [9, 6, 6, 1], [9, 7, 4, 1, 1], [9, 7, 5, 1], [9, 7, 6], [9, 8, 4, 1], [9, 8, 5], [9, 9, 3, 1], [9, 9, 4], [10, 5, 5, 1, 1], [10, 6, 4, 1, 1], [10, 6, 5, 1], [10, 6, 6], [10, 7, 4, 1], [10, 7, 5], [10, 8, 3, 1], [10, 8, 4], [10, 9, 3], [10, 10, 1, 1], [11, 5, 5, 1], [11, 6, 4, 1], [11, 6, 5], [11, 7, 3, 1], [11, 7, 4], [11, 8, 3], [11, 9, 1, 1], [11, 10, 1], [11, 11], [12, 5, 4, 1], [12, 5, 5], [12, 6, 3, 1], [12, 6, 4], [12, 7, 3], [12, 8, 1, 1], [12, 9, 1], [12, 10], [13, 5, 3, 1], [13, 5, 4], [13, 6, 3], [13, 7, 1, 1], [13, 8, 1], [13, 9], [14, 4, 4], [14, 5, 3], [14, 6, 1, 1], [14, 7, 1], [14, 8], [15, 4, 3], [15, 5, 1, 1], [15, 6, 1], [15, 7], [16, 4, 1, 1], [16, 5, 1], [16, 6], [17, 4, 1], [17, 5], [18, 3, 1], [18, 4], [19, 3], [20, 1, 1], [21, 1], [22]], [[8, 7, 5, 3], [8, 8, 4, 3], [8, 8, 5, 1, 1], [9, 6, 5, 3], [9, 6, 6, 1, 1], [9, 7, 4, 3], [9, 7, 5, 1, 1], [9, 7, 6, 1], [9, 8, 4, 1, 1], [9, 8, 5, 1], [9, 8, 6], [9, 9, 4, 1], [9, 9, 5], [10, 6, 4, 3], [10, 6, 5, 1, 1], [10, 6, 6, 1], [10, 7, 4, 1, 1], [10, 7, 5, 1], [10, 7, 6], [10, 8, 4, 1], [10, 8, 5], [10, 9, 3, 1], [10, 9, 4], [10, 10, 3], [11, 5, 5, 1, 1], [11, 6, 4, 1, 1], [11, 6, 5, 1], [11, 6, 6], [11, 7, 4, 1], [11, 7, 5], [11, 8, 3, 1], [11, 8, 4], [11, 9, 3], [11, 10, 1, 1], [11, 11, 1], [12, 5, 5, 1], [12, 6, 4, 1], [12, 6, 5], [12, 7, 3, 1], [12, 7, 4], [12, 8, 3], [12, 9, 1, 1], [12, 10, 1], [12, 11], [13, 5, 4, 1], [13, 5, 5], [13, 6, 3, 1], [13, 6, 4], [13, 7, 3], [13, 8, 1, 1], [13, 9, 1], [13, 10], [14, 5, 3, 1], [14, 5, 4], [14, 6, 3], [14, 7, 1, 1], [14, 8, 1], [14, 9], [15, 4, 4], [15, 5, 3], [15, 6, 1, 1], [15, 7, 1], [15, 8], [16, 4, 3], [16, 5, 1, 1], [16, 6, 1], [16, 7], [17, 4, 1, 1], [17, 5, 1], [17, 6], [18, 4, 1], [18, 5], [19, 3, 1], [19, 4], [20, 3], [21, 1, 1], [22, 1], [23]], [[8, 7, 5, 3, 1], [8, 8, 4, 4], [8, 8, 5, 3], [9, 6, 6, 3], [9, 7, 4, 4], [9, 7, 5, 3], [9, 7, 6, 1, 1], [9, 8, 4, 3], [9, 8, 5, 1, 1], [9, 8, 6, 1], [9, 9, 4, 1, 1], [9, 9, 5, 1], [9, 9, 6], [10, 6, 5, 3], [10, 6, 6, 1, 1], [10, 7, 4, 3], [10, 7, 5, 1, 1], [10, 7, 6, 1], [10, 7, 7], [10, 8, 4, 1, 1], [10, 8, 5, 1], [10, 8, 6], [10, 9, 4, 1], [10, 9, 5], [10, 10, 3, 1], [10, 10, 4], [11, 6, 4, 3], [11, 6, 5, 1, 1], [11, 6, 6, 1], [11, 7, 4, 1, 1], [11, 7, 5, 1], [11, 7, 6], [11, 8, 4, 1], [11, 8, 5], [11, 9, 3, 1], [11, 9, 4], [11, 10, 3], [11, 11, 1, 1], [12, 5, 5, 1, 1], [12, 6, 4, 1, 1], [12, 6, 5, 1], [12, 6, 6], [12, 7, 4, 1], [12, 7, 5], [12, 8, 3, 1], [12, 8, 4], [12, 9, 3], [12, 10, 1, 1], [12, 11, 1], [12, 12], [13, 5, 5, 1], [13, 6, 4, 1], [13, 6, 5], [13, 7, 3, 1], [13, 7, 4], [13, 8, 3], [13, 9, 1, 1], [13, 10, 1], [13, 11], [14, 5, 4, 1], [14, 5, 5], [14, 6, 3, 1], [14, 6, 4], [14, 7, 3], [14, 8, 1, 1], [14, 9, 1], [14, 10], [15, 5, 3, 1], [15, 5, 4], [15, 6, 3], [15, 7, 1, 1], [15, 8, 1], [15, 9], [16, 4, 4], [16, 5, 3], [16, 6, 1, 1], [16, 7, 1], [16, 8], [17, 4, 3], [17, 5, 1, 1], [17, 6, 1], [17, 7], [18, 4, 1, 1], [18, 5, 1], [18, 6], [19, 4, 1], [19, 5], [20, 3, 1], [20, 4], [21, 3], [22, 1, 1], [23, 1], [24]], [[8, 8, 5, 3, 1], [9, 6, 6, 3, 1], [9, 7, 5, 3, 1], [9, 7, 5, 4], [9, 7, 6, 3], [9, 8, 4, 4], [9, 8, 5, 3], [9, 8, 6, 1, 1], [9, 9, 4, 3], [9, 9, 5, 1, 1], [9, 9, 6, 1], [10, 6, 6, 3], [10, 7, 4, 4], [10, 7, 5, 3], [10, 7, 6, 1, 1], [10, 7, 7, 1], [10, 8, 4, 3], [10, 8, 5, 1, 1], [10, 8, 6, 1], [10, 8, 7], [10, 9, 4, 1, 1], [10, 9, 5, 1], [10, 9, 6], [10, 10, 4, 1], [10, 10, 5], [11, 6, 5, 3], [11, 6, 6, 1, 1], [11, 7, 4, 3], [11, 7, 5, 1, 1], [11, 7, 6, 1], [11, 7, 7], [11, 8, 4, 1, 1], [11, 8, 5, 1], [11, 8, 6], [11, 9, 4, 1], [11, 9, 5], [11, 10, 3, 1], [11, 10, 4], [11, 11, 3], [12, 6, 4, 3], [12, 6, 5, 1, 1], [12, 6, 6, 1], [12, 7, 4, 1, 1], [12, 7, 5, 1], [12, 7, 6], [12, 8, 4, 1], [12, 8, 5], [12, 9, 3, 1], [12, 9, 4], [12, 10, 3], [12, 11, 1, 1], [12, 12, 1], [13, 5, 5, 1, 1], [13, 6, 4, 1, 1], [13, 6, 5, 1], [13, 6, 6], [13, 7, 4, 1], [13, 7, 5], [13, 8, 3, 1], [13, 8, 4], [13, 9, 3], [13, 10, 1, 1], [13, 11, 1], [13, 12], [14, 5, 5, 1], [14, 6, 4, 1], [14, 6, 5], [14, 7, 3, 1], [14, 7, 4], [14, 8, 3], [14, 9, 1, 1], [14, 10, 1], [14, 11], [15, 5, 4, 1], [15, 5, 5], [15, 6, 3, 1], [15, 6, 4], [15, 7, 3], [15, 8, 1, 1], [15, 9, 1], [15, 10], [16, 5, 3, 1], [16, 5, 4], [16, 6, 3], [16, 7, 1, 1], [16, 8, 1], [16, 9], [17, 4, 4], [17, 5, 3], [17, 6, 1, 1], [17, 7, 1], [17, 8], [18, 4, 3], [18, 5, 1, 1], [18, 6, 1], [18, 7], [19, 4, 1, 1], [19, 5, 1], [19, 6], [20, 4, 1], [20, 5], [21, 3, 1], [21, 4], [22, 3], [23, 1, 1], [24, 1], [25]], [[9, 7, 5, 4, 1], [9, 7, 6, 3, 1], [9, 7, 6, 4], [9, 8, 5, 3, 1], [9, 8, 5, 4], [9, 8, 6, 3], [9, 9, 4, 4], [9, 9, 5, 3], [9, 9, 6, 1, 1], [10, 6, 6, 3, 1], [10, 7, 5, 3, 1], [10, 7, 5, 4], [10, 7, 6, 3], [10, 7, 7, 1, 1], [10, 8, 4, 4], [10, 8, 5, 3], [10, 8, 6, 1, 1], [10, 8, 7, 1], [10, 9, 4, 3], [10, 9, 5, 1, 1], [10, 9, 6, 1], [10, 9, 7], [10, 10, 4, 1, 1], [10, 10, 5, 1], [10, 10, 6], [11, 6, 6, 3], [11, 7, 4, 4], [11, 7, 5, 3], [11, 7, 6, 1, 1], [11, 7, 7, 1], [11, 8, 4, 3], [11, 8, 5, 1, 1], [11, 8, 6, 1], [11, 8, 7], [11, 9, 4, 1, 1], [11, 9, 5, 1], [11, 9, 6], [11, 10, 4, 1], [11, 10, 5], [11, 11, 3, 1], [11, 11, 4], [12, 6, 5, 3], [12, 6, 6, 1, 1], [12, 7, 4, 3], [12, 7, 5, 1, 1], [12, 7, 6, 1], [12, 7, 7], [12, 8, 4, 1, 1], [12, 8, 5, 1], [12, 8, 6], [12, 9, 4, 1], [12, 9, 5], [12, 10, 3, 1], [12, 10, 4], [12, 11, 3], [12, 12, 1, 1], [13, 6, 4, 3], [13, 6, 5, 1, 1], [13, 6, 6, 1], [13, 7, 4, 1, 1], [13, 7, 5, 1], [13, 7, 6], [13, 8, 4, 1], [13, 8, 5], [13, 9, 3, 1], [13, 9, 4], [13, 10, 3], [13, 11, 1, 1], [13, 12, 1], [13, 13], [14, 5, 5, 1, 1], [14, 6, 4, 1, 1], [14, 6, 5, 1], [14, 6, 6], [14, 7, 4, 1], [14, 7, 5], [14, 8, 3, 1], [14, 8, 4], [14, 9, 3], [14, 10, 1, 1], [14, 11, 1], [14, 12], [15, 5, 5, 1], [15, 6, 4, 1], [15, 6, 5], [15, 7, 3, 1], [15, 7, 4], [15, 8, 3], [15, 9, 1, 1], [15, 10, 1], [15, 11], [16, 5, 4, 1], [16, 5, 5], [16, 6, 3, 1], [16, 6, 4], [16, 7, 3], [16, 8, 1, 1], [16, 9, 1], [16, 10], [17, 5, 3, 1], [17, 5, 4], [17, 6, 3], [17, 7, 1, 1], [17, 8, 1], [17, 9], [18, 4, 4], [18, 5, 3], [18, 6, 1, 1], [18, 7, 1], [18, 8], [19, 4, 3], [19, 5, 1, 1], [19, 6, 1], [19, 7], [20, 4, 1, 1], [20, 5, 1], [20, 6], [21, 4, 1], [21, 5], [22, 3, 1], [22, 4], [23, 3], [24, 1, 1], [25, 1], [26]], [[9, 7, 6, 4, 1], [9, 8, 5, 4, 1], [9, 8, 6, 3, 1], [9, 8, 6, 4], [9, 9, 5, 3, 1], [9, 9, 5, 4], [9, 9, 6, 3], [10, 7, 5, 4, 1], [10, 7, 6, 3, 1], [10, 7, 6, 4], [10, 7, 7, 3], [10, 8, 5, 3, 1], [10, 8, 5, 4], [10, 8, 6, 3], [10, 8, 7, 1, 1], [10, 9, 4, 4], [10, 9, 5, 3], [10, 9, 6, 1, 1], [10, 9, 7, 1], [10, 10, 4, 3], [10, 10, 5, 1, 1], [10, 10, 6, 1], [10, 10, 7], [11, 6, 6, 3, 1], [11, 7, 5, 3, 1], [11, 7, 5, 4], [11, 7, 6, 3], [11, 7, 7, 1, 1], [11, 8, 4, 4], [11, 8, 5, 3], [11, 8, 6, 1, 1], [11, 8, 7, 1], [11, 8, 8], [11, 9, 4, 3], [11, 9, 5, 1, 1], [11, 9, 6, 1], [11, 9, 7], [11, 10, 4, 1, 1], [11, 10, 5, 1], [11, 10, 6], [11, 11, 4, 1], [11, 11, 5], [12, 6, 6, 3], [12, 7, 4, 4], [12, 7, 5, 3], [12, 7, 6, 1, 1], [12, 7, 7, 1], [12, 8, 4, 3], [12, 8, 5, 1, 1], [12, 8, 6, 1], [12, 8, 7], [12, 9, 4, 1, 1], [12, 9, 5, 1], [12, 9, 6], [12, 10, 4, 1], [12, 10, 5], [12, 11, 3, 1], [12, 11, 4], [12, 12, 3], [13, 6, 5, 3], [13, 6, 6, 1, 1], [13, 7, 4, 3], [13, 7, 5, 1, 1], [13, 7, 6, 1], [13, 7, 7], [13, 8, 4, 1, 1], [13, 8, 5, 1], [13, 8, 6], [13, 9, 4, 1], [13, 9, 5], [13, 10, 3, 1], [13, 10, 4], [13, 11, 3], [13, 12, 1, 1], [13, 13, 1], [14, 6, 4, 3], [14, 6, 5, 1, 1], [14, 6, 6, 1], [14, 7, 4, 1, 1], [14, 7, 5, 1], [14, 7, 6], [14, 8, 4, 1], [14, 8, 5], [14, 9, 3, 1], [14, 9, 4], [14, 10, 3], [14, 11, 1, 1], [14, 12, 1], [14, 13], [15, 5, 5, 1, 1], [15, 6, 4, 1, 1], [15, 6, 5, 1], [15, 6, 6], [15, 7, 4, 1], [15, 7, 5], [15, 8, 3, 1], [15, 8, 4], [15, 9, 3], [15, 10, 1, 1], [15, 11, 1], [15, 12], [16, 5, 5, 1], [16, 6, 4, 1], [16, 6, 5], [16, 7, 3, 1], [16, 7, 4], [16, 8, 3], [16, 9, 1, 1], [16, 10, 1], [16, 11], [17, 5, 4, 1], [17, 5, 5], [17, 6, 3, 1], [17, 6, 4], [17, 7, 3], [17, 8, 1, 1], [17, 9, 1], [17, 10], [18, 5, 3, 1], [18, 5, 4], [18, 6, 3], [18, 7, 1, 1], [18, 8, 1], [18, 9], [19, 4, 4], [19, 5, 3], [19, 6, 1, 1], [19, 7, 1], [19, 8], [20, 4, 3], [20, 5, 1, 1], [20, 6, 1], [20, 7], [21, 4, 1, 1], [21, 5, 1], [21, 6], [22, 4, 1], [22, 5], [23, 3, 1], [23, 4], [24, 3], [25, 1, 1], [26, 1], [27]], [[9, 7, 6, 4, 1, 1], [9, 8, 6, 4, 1], [9, 9, 5, 4, 1], [9, 9, 5, 5], [9, 9, 6, 3, 1], [9, 9, 6, 4], [10, 7, 6, 4, 1], [10, 7, 7, 3, 1], [10, 7, 7, 4], [10, 8, 5, 4, 1], [10, 8, 5, 5], [10, 8, 6, 3, 1], [10, 8, 6, 4], [10, 8, 7, 3], [10, 9, 5, 3, 1], [10, 9, 5, 4], [10, 9, 6, 3], [10, 9, 7, 1, 1], [10, 10, 4, 4], [10, 10, 5, 3], [10, 10, 6, 1, 1], [10, 10, 7, 1], [11, 7, 5, 4, 1], [11, 7, 6, 3, 1], [11, 7, 6, 4], [11, 7, 7, 3], [11, 8, 5, 3, 1], [11, 8, 5, 4], [11, 8, 6, 3], [11, 8, 7, 1, 1], [11, 8, 8, 1], [11, 9, 4, 4], [11, 9, 5, 3], [11, 9, 6, 1, 1], [11, 9, 7, 1], [11, 9, 8], [11, 10, 4, 3], [11, 10, 5, 1, 1], [11, 10, 6, 1], [11, 10, 7], [11, 11, 4, 1, 1], [11, 11, 5, 1], [11, 11, 6], [12, 6, 6, 3, 1], [12, 7, 5, 3, 1], [12, 7, 5, 4], [12, 7, 6, 3], [12, 7, 7, 1, 1], [12, 8, 4, 4], [12, 8, 5, 3], [12, 8, 6, 1, 1], [12, 8, 7, 1], [12, 8, 8], [12, 9, 4, 3], [12, 9, 5, 1, 1], [12, 9, 6, 1], [12, 9, 7], [12, 10, 4, 1, 1], [12, 10, 5, 1], [12, 10, 6], [12, 11, 4, 1], [12, 11, 5], [12, 12, 3, 1], [12, 12, 4], [13, 6, 6, 3], [13, 7, 4, 4], [13, 7, 5, 3], [13, 7, 6, 1, 1], [13, 7, 7, 1], [13, 8, 4, 3], [13, 8, 5, 1, 1], [13, 8, 6, 1], [13, 8, 7], [13, 9, 4, 1, 1], [13, 9, 5, 1], [13, 9, 6], [13, 10, 4, 1], [13, 10, 5], [13, 11, 3, 1], [13, 11, 4], [13, 12, 3], [13, 13, 1, 1], [14, 6, 5, 3], [14, 6, 6, 1, 1], [14, 7, 4, 3], [14, 7, 5, 1, 1], [14, 7, 6, 1], [14, 7, 7], [14, 8, 4, 1, 1], [14, 8, 5, 1], [14, 8, 6], [14, 9, 4, 1], [14, 9, 5], [14, 10, 3, 1], [14, 10, 4], [14, 11, 3], [14, 12, 1, 1], [14, 13, 1], [14, 14], [15, 6, 4, 3], [15, 6, 5, 1, 1], [15, 6, 6, 1], [15, 7, 4, 1, 1], [15, 7, 5, 1], [15, 7, 6], [15, 8, 4, 1], [15, 8, 5], [15, 9, 3, 1], [15, 9, 4], [15, 10, 3], [15, 11, 1, 1], [15, 12, 1], [15, 13], [16, 5, 5, 1, 1], [16, 6, 4, 1, 1], [16, 6, 5, 1], [16, 6, 6], [16, 7, 4, 1], [16, 7, 5], [16, 8, 3, 1], [16, 8, 4], [16, 9, 3], [16, 10, 1, 1], [16, 11, 1], [16, 12], [17, 5, 5, 1], [17, 6, 4, 1], [17, 6, 5], [17, 7, 3, 1], [17, 7, 4], [17, 8, 3], [17, 9, 1, 1], [17, 10, 1], [17, 11], [18, 5, 4, 1], [18, 5, 5], [18, 6, 3, 1], [18, 6, 4], [18, 7, 3], [18, 8, 1, 1], [18, 9, 1], [18, 10], [19, 5, 3, 1], [19, 5, 4], [19, 6, 3], [19, 7, 1, 1], [19, 8, 1], [19, 9], [20, 4, 4], [20, 5, 3], [20, 6, 1, 1], [20, 7, 1], [20, 8], [21, 4, 3], [21, 5, 1, 1], [21, 6, 1], [21, 7], [22, 4, 1, 1], [22, 5, 1], [22, 6], [23, 4, 1], [23, 5], [24, 3, 1], [24, 4], [25, 3], [26, 1, 1], [27, 1], [28]]]
w

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [6, 5],
  [7, 3, 1],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [6, 6],
  [7, 4, 1],
  [7, 5],
  [8, 3, 1],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [6, 6, 1],
  [7, 4, 1, 1],
  [7, 5, 1],
  [7, 6],
  [8, 4, 1],
  [8, 5],
  [9, 3, 1],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [6, 6, 1, 1],
  [7, 4, 3],
  [7, 5, 1, 1],
  [7, 6, 1],
  [7, 7],
  [8, 4, 1, 1],
  [8, 5, 1],
  [8, 6],
  [9, 4, 1],
  [9, 5],
  [10, 3, 1],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [7, 6, 1, 1],
  [7, 7, 1],
 

In [20]:
w[20]

[[7, 7, 4, 1, 1],
 [8, 5, 5, 1, 1],
 [8, 6, 4, 1, 1],
 [8, 6, 5, 1],
 [8, 7, 4, 1],
 [8, 7, 5],
 [8, 8, 3, 1],
 [8, 8, 4],
 [9, 5, 5, 1],
 [9, 6, 4, 1],
 [9, 6, 5],
 [9, 7, 3, 1],
 [9, 7, 4],
 [9, 8, 3],
 [9, 9, 1, 1],
 [10, 5, 4, 1],
 [10, 5, 5],
 [10, 6, 3, 1],
 [10, 6, 4],
 [10, 7, 3],
 [10, 8, 1, 1],
 [10, 9, 1],
 [10, 10],
 [11, 5, 3, 1],
 [11, 5, 4],
 [11, 6, 3],
 [11, 7, 1, 1],
 [11, 8, 1],
 [11, 9],
 [12, 4, 4],
 [12, 5, 3],
 [12, 6, 1, 1],
 [12, 7, 1],
 [12, 8],
 [13, 4, 3],
 [13, 5, 1, 1],
 [13, 6, 1],
 [13, 7],
 [14, 4, 1, 1],
 [14, 5, 1],
 [14, 6],
 [15, 4, 1],
 [15, 5],
 [16, 3, 1],
 [16, 4],
 [17, 3],
 [18, 1, 1],
 [19, 1],
 [20]]

In [21]:
list(Partitions(5))

[[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]

In [31]:
several_virasoro([3,3,2,1],{(0,):1})

{(6, 2, 0): 25, (9,): 100, (5, 3, 0): -10}

In [30]:
several_virasoro((3,3,2,1),{(0,):1})

{(6, 2, 0): 25, (9,): 100, (5, 3, 0): -10}

In [34]:
basis1(20)

[1, [[1]], 1]
[2, [[1, 1]], 1]
[3, [[1, 1, 1]], 1]
[4, [[1, 1, 1, 1], [2, 1, 1]], 2]
[5, [[1, 1, 1, 1, 1], [2, 1, 1, 1]], 2]
[6, [[1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1], [2, 2, 1, 1]], 3]
[7, [[1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1], [3, 1, 1, 1, 1]], 4]
[8, [[1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1], [2, 2, 2, 1, 1], [3, 1, 1, 1, 1, 1]], 5]
[9, [[1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1]], 6]
[10, [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1], [3, 2, 2, 1, 1, 1]], 8]
[11, [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 2, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1, 1, 1, 1], [3, 2, 2, 2, 1, 1]], 9]
[1